In [92]:
# !pip install sentence_transformers
import requests
from bs4 import BeautifulSoup
import torch
from sentence_transformers import SentenceTransformer

In [93]:
def get_sentences_for_claim(wikipedia_headings):

    url = 'https://en.wikipedia.org/w/api.php'
    sentences = []

    for subject in wikipedia_headings:

        subject = subject.replace("-LRB-", "(")
        subject = subject.replace("-RRB-", ")")
        try:
            params = {
                        'action': 'parse',
                        'page': subject,
                        'format': 'json',
                        'prop':'text',
                        'redirects':''
                    }
        
        
            response = requests.get(url, params=params)
            data = response.json()
            
            raw_html = data['parse']['text']['*']
            soup = BeautifulSoup(raw_html,'html.parser')
            soup.find_all('p')
            text = ''
            
            for p in soup.find_all('p'):
                text += p.text
            
            sentence_list = text.split(".")
            sentences.extend(sentence_list)
        except:
            print("Following subject not recognized by the wikipedia API", subject)
            continue

    return sentences

In [94]:
query = "Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League."
documents = ['Colin_Kaepernick', 'National_League_-LRB-English_football-RRB-', 'National_Football_League']

sentences = get_sentences_for_claim(documents)

model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(sentences)
query_embeddings = model.encode(query)

similarity = torch.cosine_similarity(
    torch.from_numpy(query_embeddings),
    torch.from_numpy(sentence_embeddings)
)

indices = torch.argsort(similarity, descending=True).numpy()[:5]
output = [sentences[i] for i in indices]

for out in output:
    print(out)

 During the 2013 season, his first full season as a starter, Kaepernick helped the 49ers reach the NFC Championship Game
[76] He returned to the starting lineup the following week and threw for 183 yards and two touchdowns in the 49ers' 41–13 loss to the Atlanta Falcons
[78] For the 2016 NFL season, Kaepernick played twelve games and ended the season with 2,241 passing yards, sixteen passing touchdowns, four interceptions and added 468 rushing yards and two rushing touchdowns
 Kaepernick began his professional football career as a backup quarterback to Alex Smith, and became the 49ers' starter in the middle of the 2012 season after Smith suffered a concussion
[51]
In the season opener of the 2013 season against the Green Bay Packers, Kaepernick threw for a career-high 412 yards and three touchdowns, the first 400-yard game by a 49ers quarterback since Tim Rattay on October 10, 2004
